In [4]:
import pandas as pd
import numpy as np

In [5]:
# Read in the transactions csv containing data on customer stock transactions
df = pd.read_csv('accounts.csv')

In [6]:
# Rename the columns currently in the data frame in order to remove the space between words in the column titles
df = df.rename(columns = {'First Name' : 'FirstName', 'Last Name' : 'LastName', 'Account ID' : 'AccountID', 'Account Type' : 'AccountType', 
                          'Mailing Address' : 'MailingAddress', 'Account Creation Date' : 'AccountCreationDate', 'Date of Birth' : 'DateOfBirth', 
                          'Account Status' : 'AccountStatus', 'Risk Profile' : 'RiskProfile', 
                         'Risk Tolerance' : 'RiskTolerance', 'Income Bracket' : 'IncomeBracket', 'Employment Status' : 'EmploymentStatus', 
                         'Account Opened Date' : 'AccountOpenedDate'})

In [7]:
# Make the columns where case doesn't matter upper case in order to standardize the row entries
df[['FirstName', 'LastName', 'AccountType', 'RiskTolerance', 'EmploymentStatus', 'IncomeBracket']] = df[['FirstName', 'LastName', 'AccountType', 'RiskTolerance', 'EmploymentStatus', 'IncomeBracket']].astype(str).apply(lambda col: col.str.upper())

In [8]:
# Drop any rows that are exactly the same
df = df.drop_duplicates()

In [9]:
# Check to see if there are any rows with the same 'AccountID' because if there are, we must remove one because we want 'AccountID'
# to be our primary key in the table
df[df['AccountID'].duplicated()]

,AccountID,FirstName,LastName,SSN,Email,Phone,DateOfBirth,MailingAddress,AccountType,RiskTolerance,IncomeBracket,EmploymentStatus,AccountOpenedDate


In [10]:
# Make the accountID be only yhe number within because it is unique
df['AccountID'] = df['AccountID'].str.split('-').str[1] + df['AccountID'].str.split('-').str[2]

In [11]:
# SPlit 'PhoneNumber' into the usual phone number and extension number if needed
df['PhoneNumber'] = df['Phone'].str.split('x').str[0]
df['ExtensionNumber'] = df['Phone'].str.split('x').str[1]
df = df.drop('Phone', axis=1)

In [12]:
# Combine the first and last names to get a name column in order to clean the names by removing titles such as 'Mr.' and ending titles as well
df['Name'] = df['FirstName'] + ' ' + df['LastName']
formal_titles_df = df['Name'][df['Name'].str.split('.').str[1].isna() == 0]
df['Name'][df['Name'].str.split('.').str[1].isna() == 0] = (formal_titles_df.str.split('.').str[1].str.split(' ').str[1] + ' ' + 
formal_titles_df.str.split('.').str[1].str.split(' ').str[2])

In [13]:
# Set 'FirstName' and 'LastName' to the new cleaned versions and drop the column 'Name' which we only used for cleaning
df['FirstName'] = df['Name'].str.split(' ').str[0]
df['LastName'] = df['Name'].str.split(' ').str[1]
df = df.drop('Name', axis=1)

In [14]:
# Standardize the 'PhoneNumbers' column 
df['PhoneNumber'] = df['PhoneNumber'].str.replace('.', '').str.replace('(', '').str.replace(')', '').str.replace('-', '')

In [15]:
# Delete the whitespace before and after every entry in every column where the quantity is of type 'object'
for col in df.select_dtypes(include='object'):
    df[col] = df[col].str.strip()

In [16]:
# Replace the different entries for empty with None in order to make it simpler when importing for sql use
df = df.replace(['NAN', 'NaN', np.nan], None)

In [17]:
# Clean all of the columns dealing with dates in order to convert them to the format yyyy-mm-dd

columns = ['DateOfBirth', 'AccountOpenedDate']

for column in columns:

    # If the date appears as month dd, yyyy then change the month spelled out to the numeric mm it corresponds to
    df[column][df[column].str.split('-').str[1].isna() == 0] = (
        df[column][df[column].str.split('-').str[1].isna() == 0].str.replace('Jan', '01').str.replace('Feb', '02')
        .str.replace('Mar', '03').str.replace('Apr', '04').str.replace('May', '05').str.replace('Jun', '06').str.replace('Jul', '07')
        .str.replace('Aug', '08').str.replace('Sep', '09').str.replace('Oct', '10').str.replace('Nov', '11').str.replace('Dec', '12')
    )

    # Define month_dd_yyyy_dates_dash as the dates that appear as day-abbreviated_month-year
    month_dd_yyyy_dates_dash_df = df[column][df[column].str.split('-').str[1].isna() == 0]
    # Set those dates that appear as day-abbreviated_month-yyyy to be in the form yyyy-mm-dd
    df[column][df[column].str.split('-').str[1].isna() == 0] = (
        month_dd_yyyy_dates_dash_df.str.split('-').str[2] + '-' + month_dd_yyyy_dates_dash_df.str.split('-').str[1] + '-' +
        month_dd_yyyy_dates_dash_df.str.split('-').str[0]
    )

    # Define mm_dd_yyyy_dates_slash as those dates in the 'DateCompleted' column that look like mm/dd/yyyy
    mm_dd_yyyy_dates_slash_df = df[column][df[column].str.split('/').str[1].isna() == 0]
    # Change the dates that look like mm/dd/yyyy to be yyyy-mm-dd
    df[column][df[column].str.split('/').str[1].isna() == 0] = (
        mm_dd_yyyy_dates_slash_df.str.split("/").str[2] + '-' + mm_dd_yyyy_dates_slash_df.str.split("/").str[0] + '-' + mm_dd_yyyy_dates_slash_df.str.split("/").str[1])

    # Since the placeholder 0 is missing from some of these dates, we fill them in in order to get two digits for day and month in each date
    df[column] = df[column].str.split('-').str[0] + '-' + df[column].str.split('-').str[1].str.zfill(2) + '-' + df[column].str.split('-').str[2].str.zfill(2)

In [18]:
df.to_csv(r'C:\Users\beanw\OneDrive\Desktop\finance_project\cleaned_accounts.csv', index=False) 